In [1]:
import glob
from utils.analysis import Extrapolation

In [2]:
TAG = "pku"
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
data_babies = [baby for baby in babies if "data" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = list(set(babies) - set(data_babies + sig_babies))
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/vbswh/output_pku/Run2/data.root']
['../analysis/studies/vbswh/output_pku/Run2/VBSWH_mkW.root']


['../analysis/studies/vbswh/output_pku/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_pku/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_pku/Run2/WJets.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_pku/Run2/TTX.root',
 '../analysis/studies/vbswh/output_pku/Run2/VH.root',
 '../analysis/studies/vbswh/output_pku/Run2/SingleTop.root']

In [3]:
vbswh = Extrapolation(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=["xsec_sf", "lep_sf", "btag_sf"],
    plots_dir=f"/home/users/jguiang/public_html/onelep_plots/{TAG}/val"
)
vbswh.df["presel_noVBS_noBVeto"] = vbswh.df.eval(
    "hbbjet_score > 0.3"
)
vbswh.df["presel_noVBS"] = vbswh.df.eval(
    "passes_bveto and hbbjet_score > 0.3"
)
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)

In [4]:
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,202.10857867741984,3.6103742602363678,11.230803607061898,1.3966806081207974,148,12.165525060596439
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,263.3382171032529,6.08436294502786,0.9246911028554529,0.45582975134082687,200,14.142135623730951
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,204.81192919775825,4.817905474605206,16.188206154606853,1.6879951206435762,181,13.45362404707371
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,140.29909364289,4.356119847562057,425.433362227746,8.888392997964642,BLINDED,BLINDED

name,extp,rel_err
AtoB,1.3029541785238123,0.02920505662163772
Bt

In [5]:
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 1500 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,9.496714295747807,0.985751744414441,2.630401572644729,0.66472704003939,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,19.69456272647612,4.096366802192926,0.0,0.0,9,3.0
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,12.567410931764416,1.336882682044473,3.94434888343905,0.8265752064164101,BLINDED,BLINDED
presel_noDetaJJ and M_jj > 600 and ST > 1500 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,6.582688826744839,1.379090062704561,120.43521728734709,4.770373319671058,BLINDED,BLINDED

name,extp,rel_err
AtoB,2.073829127964231,0.23245671203241278
BtoC,0.6381157635386129,0.23361911971118532
CtoD,0.52379037038

In [6]:
vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 2
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
print("")

vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9",
#     "presel_noDetaJJ and M_jj > 600 and LT > 400 and ST > 900 and hbbjet_score > 0.9",
    h_dir="left", v_dir="up", 
#     show_raw=True,
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,215.05965544042851,3.8488656601004028,11.230803607061898,1.3966806081207974,148,12.165525060596439
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,296.0422685547973,6.239641303962464,0.9246911028554529,0.45582975134082687,200,14.142135623730951
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,252.2708176136411,5.1522548553288505,16.188206154606853,1.6879951206435762,181,13.45362404707371
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,166.12133841634198,5.054924840478127,425.433362227746,8.888392997964642,BLINDED,BLINDED

name,extp,rel_err
AtoB,1.3765588340989459,0.0276500837274335